In [1]:
#importing libraries
import cv2
import numpy as np
from skimage import exposure, feature
from sklearn import svm
from sklearn import metrics
from skimage.feature import hog
import os



In [2]:
# Extracting HOG features from an image
def calculate_hog_features(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    fd, _ = hog(gray_image, orientations=8, pixels_per_cell=(8, 8),
                cells_per_block=(2, 2), visualize=True, block_norm='L2-Hys')
    return fd

In [3]:
#Exracting hog features
def extract_hog(main_folder_path):
    images = []
    hog_features_list = []
    labels_list = []

    # Create a mapping between emotion categories and integer labels
    emotion_label_mapping = {}
    label_counter = 1

    for emotion_category in os.listdir(main_folder_path):
        emotion_path = os.path.join(main_folder_path, emotion_category)

        if os.path.isdir(emotion_path):
            # Use integer labels
            emotion_label = label_counter

            # mapping
            emotion_label_mapping[emotion_category] = emotion_label
            label_counter += 1

            # Iterating through image files in the subfolder
            for filename in os.listdir(emotion_path):
                if filename.endswith(('.jpg', '.jpeg', '.png')):
                    img_path = os.path.join(emotion_path, filename)
                    img = cv2.imread(img_path)

                    if img is not None:
                        # Extract HOG features from the image
                        hog_features = calculate_hog_features(img)

                        # Appendong the image, HOG features, and label to the respective lists
                        images.append(img)
                        hog_features_list.append(hog_features)
                        labels_list.append(emotion_label)

    return images, hog_features_list, labels_list, emotion_label_mapping


with normalization

In [11]:
from sklearn.preprocessing import MinMaxScaler
def load_images_and_extract_hog_with_normalization(main_folder_path):
    images = []
    hog_features_list = []
    labels_list = []

    # mapping between emotion categories and integer labels
    emotion_label_mapping = {}
    label_counter = 1

    for emotion_category in os.listdir(main_folder_path):
        emotion_path = os.path.join(main_folder_path, emotion_category)
        
        if os.path.isdir(emotion_path):
            # Using integer labels
            emotion_label = label_counter

            # Updatong the mapping
            emotion_label_mapping[emotion_category] = emotion_label
            label_counter += 1

            # Iterating through image files in the subfolder
            for filename in os.listdir(emotion_path):
                if filename.endswith(('.jpg', '.jpeg', '.png')):
                    img_path = os.path.join(emotion_path, filename)
                    img = cv2.imread(img_path)

                    if img is not None:
                        # Extracting HOG features from the image
                        hog_features = calculate_hog_features(img)

                        # Append the image and label to the respective lists
                        images.append(img)
                        labels_list.append(emotion_label)

                        # Append the raw HOG features
                        hog_features_list.append(hog_features)

    # Converting the raw HOG features to NumPy array
    hog_features_array = np.array(hog_features_list)

    # Applying Min-Max scaling for feature normalization
    scaler = MinMaxScaler()
    hog_features_normalized = scaler.fit_transform(hog_features_array)

    return images, hog_features_normalized, labels_list, emotion_label_mapping

In [12]:
main_folder_path = '/kaggle/input/daataset/train/train'
images, hog_features_list, labels_list, emotion_label_mapping = extract_hog_with_normalization(main_folder_path)

# Convert the lists to NumPy arrays
images = np.array(images)
hog_features_list = np.array(hog_features_list)
labels_list = np.array(labels_list)

In [15]:
#Importing necessary libraries
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Splitting into training and validation
X_train, X_val, y_train_encoded, y_val_encoded = train_test_split(hog_features_list, labels_list, test_size=0.2, random_state=42)

# Converting emotion labels to integer labels using emotion_label_mapping
y_train_encoded = [emotion_label_mapping.get(label, label) for label in y_train_encoded]
y_val_encoded = [emotion_label_mapping.get(label, label) for label in y_val_encoded]
svm_model = SVC(kernel='poly',C=0.1)
svm_model.fit(X_train, y_train_encoded)  

# Predictions on the validation set
val_predictions = svm_model.predict(X_val)

# Evaluating the accuracy
accuracy = accuracy_score(y_val_encoded, val_predictions)
print("Validation Accuracy:", accuracy)

Validation Accuracy: 0.5898595572482742


In [16]:
#Importing libraries for cross validaton
from sklearn.model_selection import cross_val_score, KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score


# Spliting the data into features and labels
X = hog_features_list
y = labels_list

# Converting emotion labels to integer labels using emotion_label_mapping
y_encoded = [emotion_label_mapping.get(label, label) for label in y]


# Specify the number of folds for cross-validation
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

# Performing k-fold cross-validation
cross_val_scores = cross_val_score(svm_model, X, y_encoded, cv=kf, scoring='accuracy')

# cross-validation scores
print("Cross-Validation Scores:", cross_val_scores)


print("Mean Accuracy:", cross_val_scores.mean())



Cross-Validation Scores: [0.58985956 0.58081409 0.60533206 0.58319448 0.5858129 ]
Mean Accuracy: 0.5890026184241848
